In [9]:
import yfinance as yf
from datetime import datetime
import warnings

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


In [2]:
# Criar objeto do ticker
ticker_data = yf.Ticker("MSFT")

In [ ]:
#### Não Funciona ####
# # Acessar fast_info
info = ticker_data.info

In [6]:
yahoo_symbol_recommendations = ticker_data.recommendations
# yahoo_symbol_recommendations

In [8]:
# Dividendos
cashflow = ticker_data.cashflow
# cashflow

In [9]:
# Notícias
balance_sheet = ticker_data.balance_sheet
# balance_sheet

In [10]:
yahoo_symbol_sustainability = ticker_data.sustainability
# yahoo_symbol_sustainability

In [9]:
# Eventos corporativos
calendar = ticker_data.calendar
# calendar

In [10]:
# # Datas disponíveis para opções
# options_dates = ticker_data.options
# print("Datas de opções:", options_dates)

# # Obter opções para uma data específica
# options = ticker_data.option_chain(options_dates[0])
# print("Opções de compra (calls):", options.calls)
# print("Opções de venda (puts):", options.puts)

In [5]:
datetime.now()

NameError: name 'datetime' is not defined

Informações financeiras mais detalhadas da empresa.

In [12]:
# # Dados de sustentabilidade
# sustainability = ticker_data.sustainability
# print(sustainability)

Informações de ESG (Environmental, Social, Governance).

In [13]:
# # Ações em circulação
# shares = ticker_data.shares
# print("Ações em circulação:", shares)

Obtenha informações sobre ações em circulação e histórico de splits.

In [14]:
# Principais investidores institucionais
inst_holders = ticker_data.institutional_holders
# inst_holders


Dados sobre investidores institucionais e sua participação.

In [15]:
# Principais investidores institucionais
holders = ticker_data.major_holders
# holders


In [10]:
def get_yahoo_data_history(symbol : str, period : str, interval: str, start = '1900-01-01', end = datetime.now(), prepost : bool = True):
        '''
        Data collection from yahoo

        Parameters:
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
        end: str
            Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
        prepost : bool
            Include Pre and Post market data in results? Default is False
        debug: bool
            If passed as False, will suppress error message printing to console.
        '''
        yahoo_data_history = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost)
        return yahoo_data_history

history = get_yahoo_data_history(symbol="MSFT",period='1y', interval='1d', end = datetime.now(), prepost=True)

In [14]:
import requests

# Define the endpoint and your API key
url = "https://api.simplywall.st/graphql"
headers = {
    "Authorization": "Bearer sws:ZGE4YzBlNzItMTg2Ni00Y2Y5LWI3YTktYzAxODcyZGEwZjE3OmFjODUzODRhZDUzNGMwMzY=",
    "Content-Type": "application/json",  # Required for JSON body
}

# Define the GraphQL query and variables
query = """
query Companies($exchange: String!, $offset: Int!, $limit: Int!) {
  companies(exchange: $exchange, offset: $offset, limit: $limit) {
    id
    exchangeSymbol
    tickerSymbol
    name
    marketCapUSD
    primaryIndustry { name }
    secondaryIndustry { name }
    tertiaryIndustry { name }
    market { name iso2 }
    closingPrices
    statements { name title area type value outcome description state severity outcomeName }
    listings { id exchangeSymbol tickerSymbol name marketCapUSD primaryIndustry { name } secondaryIndustry { name } tertiaryIndustry { name } market { name iso2 } closingPrices }
    owners { name type sharesHeld percentOfSharesOutstanding holdingDate periodStartDate periodEndDate }
    insiderTransactions { type ownerName ownerType description tradeDateMin tradeDateMax shares priceMin priceMax transactionValue percentageSharesTraded }
    members { age name title tenure compensation }
    active classificationStatus
  }
}
"""

# Define the variables for the query
variables = {
    "exchange": "NYSE",  # Try changing to "NYSE" or another exchange
    "offset": 20,         # Try adjusting this value (e.g., 10 or 20)
    "limit": 100          # Adjust limit to fetch more results
}

# Create the request payload
payload = {
    "query": query,
    "variables": variables
}

# Make the POST request
response = requests.post(url, headers=headers, json=payload)

# Check the response
if response.status_code == 200:
    data = response.json()
    if not data:
        print("No companies found with the given parameters.")
    else:
        print("Returned Data:", data)
else:
    print(f"Error: {response.status_code}, {response.text}")


Returned Data: {'data': {'companies': []}}


In [1]:
import os
import subprocess
import requests

class TALibInstaller:
    def install_ta_lib_windows(self):
        """Automatiza a instalação do TA-Lib no Windows."""
        print("Iniciando instalação do TA-Lib no Windows...")

        # URL do arquivo .whl (atualize conforme necessário)
        talib_url = "https://github.com/cgohlke/talib-build/releases/download/v0.5.1/TA_Lib-0.5.1-cp311-cp311-win_amd64.whl"
        talib_whl = talib_url.split("/")[-1]  # Nome do arquivo

        # Caminho local para salvar o arquivo .whl
        local_path = os.path.join(os.getcwd(), "libraries")

        # Fazer o download do arquivo .whl
        if not os.path.exists(local_path):
            print(f"Baixando {talib_whl} de {talib_url}...")
            response = requests.get(talib_url, stream=True)
            if response.status_code == 200:
                with open(local_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        f.write(chunk)
                print(f"Download concluído: {local_path}")
            else:
                print(f"Erro ao baixar {talib_url}: {response.status_code}")
                return False
        else:
            print(f"O arquivo {talib_whl} já existe. Pulando o download.")

        # Instalar o arquivo .whl com pip
        print(f"Instalando {talib_whl}...")
        try:
            subprocess.check_call(["pip", "install", local_path])
            print("TA-Lib instalado com sucesso!")
        except subprocess.CalledProcessError as e:
            print(f"Erro durante a instalação: {e}")
            return False

        # Remover o arquivo .whl após a instalação (opcional)
        if os.path.exists(local_path):
            os.remove(local_path)
            print(f"Arquivo {local_path} removido após a instalação.")
        return True


# Uso
if __name__ == "__main__":
    installer = TALibInstaller()
    installer.install_ta_lib_windows()


Iniciando instalação do TA-Lib no Windows...
O arquivo TA_Lib-0.5.1-cp311-cp311-win_amd64.whl já existe. Pulando o download.
Instalando TA_Lib-0.5.1-cp311-cp311-win_amd64.whl...
TA-Lib instalado com sucesso!
Arquivo x:\Taurus\market-analyzer\backend\test\TA_Lib-0.5.1-cp311-cp311-win_amd64.whl removido após a instalação.


In [1]:
import talib
import pandas as pd

class CandlesPatterns:
    """
    Detect all candlestick patterns using TA-Lib.
    """

    def __init__(self):
        self.result_candles_history_df = pd.DataFrame(columns=['Pattern', 'Signal', 'Relevance', 'Stoploss'])
        self.result_candles_df = pd.DataFrame(columns=['Pattern', 'Signal', 'Relevance', 'Stoploss'])
        
        
    def detect_pattern(self, data: pd.DataFrame, pattern_function, pattern_name: str):
        """
        General method to detect a specific candlestick pattern.
        """
        data_pass = data

        detection = pattern_function(data_pass['Open'], data_pass['High'], data_pass['Low'], data_pass['Close'])

        non_zero_detection = detection[detection != 0]
        if not non_zero_detection.empty:
            for date, signal in non_zero_detection.items():
                # Cálculo do Stoploss baseado no padrão
                if pattern_name in [
                    "doji", "dragonfly_doji", "gravestone_doji", "engulfing",
                    "morning_star", "evening_star", "marubozu", "harami",
                    "harami_cross", "kicking", "kicking_by_length", "tasuki_gap",
                    "gap_side_by_side_white", "counter_attack", "piercing",
                    "dark_cloud_cover", "tri_star"
                ]:
                    stoploss = round(data_pass.loc[date, 'Low'],5) if signal > 0 else round(data_pass.loc[date, 'High'],5)
                elif pattern_name in [
                    "morning_doji_star", "hammer", "inverted_hammer",
                    "thrusting", "matching_low", "three_white_soldiers",
                    "three_outside", "three_stars_in_south"
                ]:
                    stoploss = round(data_pass.loc[date, 'Low'],5)
                elif pattern_name in [
                    "evening_doji_star", "hanging_man", "shooting_star",
                    "on_neck", "in_neck", "three_black_crows",
                    "three_inside", "advance_block", "stalled_pattern"
                ]:
                    stoploss = round(data_pass.loc[date, 'High'],5)
                else:
                    stoploss = None

                new_entry = pd.DataFrame({
                    'Pattern': [pattern_name],
                    'Signal': [signal],
                    'Relevance': ['Flat'],
                    'Stoploss': [stoploss],
                }, index=[date])

                self.result_candles_history_df = pd.concat([self.result_candles_history_df, new_entry], ignore_index=False)

                self.result_candles_df = self.result_candles_df[self.result_candles_df['Pattern'] != pattern_name]
                self.result_candles_df = pd.concat([self.result_candles_df, new_entry], ignore_index=False)

        return detection

    def doji(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLDOJI, "doji")

    def dragonfly_doji(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLDRAGONFLYDOJI, "dragonfly_doji")

    def gravestone_doji(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLGRAVESTONEDOJI, "gravestone_doji")

    def engulfing(self, data: pd.DataFrame):
        return self.detect_pattern(data, talib.CDLENGULFING, "engulfing")

    def morning_star(self, data: pd.DataFrame):
        return self.detect_pattern(data, talib.CDLMORNINGSTAR, "morning_star")

    def evening_star(self, data: pd.DataFrame):
        return self.detect_pattern(data, talib.CDLEVENINGSTAR, "evening_star")

    def morning_doji_star(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLMORNINGDOJISTAR, "morning_doji_star")

    def evening_doji_star(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLEVENINGDOJISTAR, "evening_doji_star")

    def hammer(self, data: pd.DataFrame):
        return self.detect_pattern(data, talib.CDLHAMMER, "hammer")

    def inverted_hammer(self, data: pd.DataFrame):
        return self.detect_pattern(data, talib.CDLINVERTEDHAMMER, "inverted_hammer")

    def hanging_man(self, data: pd.DataFrame):
        return self.detect_pattern(data, talib.CDLHANGINGMAN, "hanging_man")

    def shooting_star(self, data: pd.DataFrame):
        return self.detect_pattern(data, talib.CDLSHOOTINGSTAR, "shooting_star")

    def marubozu(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLMARUBOZU, "marubozu")

    def harami(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLHARAMI, "harami")

    def harami_cross(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLHARAMICROSS, "harami_cross")

    def spinning_top(self, data: pd.DataFrame):
        '''
        NEED FIBONACCI
        '''
        return self.detect_pattern(data, talib.CDLSPINNINGTOP, "spinning_top")

    def kicking(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLKICKING, "kicking")

    def kicking_by_length(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLKICKINGBYLENGTH, "kicking_by_length")

    def tasuki_gap(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLTASUKIGAP, "tasuki_gap")

    def gap_side_by_side_white(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLGAPSIDESIDEWHITE, "gap_side_by_side_white")

    def counter_attack(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLCOUNTERATTACK, "counter_attack")

    def piercing(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLPIERCING, "piercing")

    def dark_cloud_cover(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLDARKCLOUDCOVER, "dark_cloud_cover")

    def tri_star(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLTRISTAR, "tri_star")

    def on_neck(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLONNECK, "on_neck")

    def in_neck(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLINNECK, "in_neck")

    def thrusting(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLTHRUSTING, "thrusting")

    def matching_low(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLMATCHINGLOW, "matching_low")

    def three_black_crows(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDL3BLACKCROWS, "three_black_crows")

    def three_white_soldiers(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDL3WHITESOLDIERS, "three_white_soldiers")

    def three_inside(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDL3INSIDE, "three_inside")

    def three_outside(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDL3OUTSIDE, "three_outside")

    def three_stars_in_south(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDL3STARSINSOUTH, "three_stars_in_south")

    def advance_block(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLADVANCEBLOCK, "advance_block")

    def stalled_pattern(self, data: pd.DataFrame):
        # takeprofit = None
        return self.detect_pattern(data, talib.CDLSTALLEDPATTERN, "stalled_pattern")

    # def abandoned_baby(self, data: pd.DataFrame):
    #     return self.detect_pattern(data, talib.CDLABANDONEDBABY, "abandoned_baby")

    # def unique_3_river(self, data: pd.DataFrame):
    #     return self.detect_pattern(data, talib.CDLUNIQUE3RIVER, "unique_3_river")

    # def belt_hold(self, data: pd.DataFrame):
    #     return self.detect_pattern(data, talib.CDLBELTHOLD, "belt_hold")

    # def separating_lines(self, data: pd.DataFrame):
    #     return self.detect_pattern(data, talib.CDLSEPARATINGLINES, "Separating Lines")

    # def upside_gap_two_crows(self, data: pd.DataFrame):
    #     return self.detect_pattern(data, talib.CDLUPSIDEGAP2CROWS, "Upside Gap Two Crows")


In [2]:
import yfinance as yf

start, end = '2015-01-01', '2024-07-05'

data = yf.Ticker('SPY').history(period='1y', interval='1d', start=start, end=end)

cm = CandlesPatterns()

# Loop para detectar padrões
for candle_function in dir(cm):
    if (not candle_function.startswith("__") and 
        callable(getattr(cm, candle_function)) and 
        candle_function != "detect_pattern"):
        pattern_function = getattr(cm, candle_function)
        try:
            candle_result = pattern_function(data)
        except Exception as e:
            print(f"Error detecting pattern {candle_function}: {e}")

# Verificar padrões detectados e valores de stoploss
print("\nAll Detected Patterns with Stoploss:")
cm.result_candles_df


C:\Users\joaom\AppData\Local\Temp\ipykernel_22476\45596638.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  self.result_candles_history_df = pd.concat([self.result_candles_history_df, new_entry], ignore_index=False)
C:\Users\joaom\AppData\Local\Temp\ipykernel_22476\45596638.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.result_candles_history_df = pd.concat([self.result_candles_history_df, new_entry], ignore_index=False)
C:\Users\joaom\AppData\Local\Temp\ipykernel_22476\45596638.py:59: FutureWarning: The behav


All Detected Patterns with Stoploss:


,Pattern,Signal,Relevance,Stoploss
2024-06-18 00:00:00-04:00,advance_block,-100,Flat,543.35483
2023-08-18 00:00:00-04:00,counter_attack,100,Flat,424.30297
2022-07-22 00:00:00-04:00,dark_cloud_cover,-100,Flat,385.83253
2024-07-01 00:00:00-04:00,doji,100,Flat,539.04680
2024-07-01 00:00:00-04:00,dragonfly_doji,100,Flat,539.04680
2024-07-02 00:00:00-04:00,engulfing,100,Flat,540.16963
2023-07-20 00:00:00-04:00,evening_doji_star,-100,Flat,445.94876
2024-02-26 00:00:00-05:00,evening_star,-100,Flat,502.30678
2023-11-24 00:00:00-05:00,gap_side_by_side_white,100,Flat,447.15796
2024-03-28 00:00:00-04:00,gravestone_doji,100,Flat,517.76285


In [ ]:
import talib
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed


class ParamsOptimization():
    """
    A class for optimizing technical indicators' parameters and evaluating strategy performance.
    """

    def __init__(self, ticker, market_type, timeframe):
        self.ticker = ticker
        self.market_type = market_type
        self.timeframe = timeframe
        self.best_params

    def fetch_data(self, ticker):
        """
        Simulate fetching market data for the given ticker.
        Replace this method with the actual data retrieval logic.
        """
        # Simulate historical data
        np.random.seed(42)
        size = 1000  # Simulated number of rows
        data = pd.DataFrame({
            "close": np.random.uniform(100, 200, size),
            "volume": np.random.randint(1000, 5000, size)
        })
        return data

    def optimize(self):
        """
        Run optimization for all strategies.
        """
        tickers = [self.ticker] if isinstance(self.ticker, str) else self.ticker
        all_results = []

        for ticker in tickers:
            data = self.fetch_data(ticker)
            crossover_params = self.optimize_crossover(data, ticker)
            bbands_params = self.optimize_bbands(data, ticker)
            all_results.append(pd.concat([crossover_params, bbands_params], axis=1))

        self.best_params = pd.concat(all_results, ignore_index=True)
        return self.best_params

    def optimize_crossover(self, data, ticker):
        """
        Optimize EMA crossover strategy.
        """
        ema1_periods = [10, 15, 20]
        ema2_periods = [25, 30, 50]
        ema3_periods = [100, 150, 200]

        combinations = list(itertools.product(ema1_periods, ema2_periods, ema3_periods))

        # Parallel execution for speed
        results = Parallel(n_jobs=-1)(delayed(self.simulate_crossover)(
            data, ticker, l1, l2, l3) for l1, l2, l3 in tqdm(combinations, desc="Optimizing EMA Crossover"))

        # Combine results into a DataFrame
        results_df = pd.DataFrame(results)
        best_result = results_df.loc[results_df['Expectancy'].idxmax()]

        return best_result

    def simulate_crossover(self, data, ticker, l1, l2, l3):
        """
        Simulate crossover strategy and calculate metrics.
        """
        # Calculate EMAs
        data['ema1'] = talib.EMA(data['close'], timeperiod=l1)
        data['ema2'] = talib.EMA(data['close'], timeperiod=l2)
        data['ema3'] = talib.EMA(data['close'], timeperiod=l3)

        # Generate signals
        data['signal'] = np.where((data['ema1'] > data['ema2']) & (data['ema2'] > data['ema3']), 1,
                                  np.where((data['ema1'] < data['ema2']) & (data['ema2'] < data['ema3']), -1, 0))
        data['returns'] = data['close'].pct_change() * data['signal'].shift(1)

        # Calculate metrics
        sharpe = self.calculate_sharpe(data['returns'])
        max_drawdown = self.calculate_max_drawdown(data['returns'])
        expectancy = self.calculate_expectancy(data['returns'])

        return {
            'Ticker': ticker,
            'Best_EMA1': l1,
            'Best_EMA2': l2,
            'Best_EMA3': l3,
            'Sharpe': sharpe,
            'Max_Drawdown': max_drawdown,
            'Expectancy': expectancy
        }

    def optimize_bbands(self, data, ticker):
        """
        Optimize Bollinger Bands strategy.
        """
        sma_periods = [10, 15, 20]
        std_devs = [1, 2, 3]

        combinations = list(itertools.product(sma_periods, std_devs))

        # Parallel execution for speed
        results = Parallel(n_jobs=-1)(delayed(self.simulate_bbands)(
            data, ticker, period, std) for period, std in tqdm(combinations, desc="Optimizing Bollinger Bands"))

        # Combine results into a DataFrame
        results_df = pd.DataFrame(results)
        best_result = results_df.loc[results_df['Expectancy'].idxmax()]

        return best_result

    def simulate_bbands(self, data, ticker, period, std):
        """
        Simulate Bollinger Bands strategy and calculate metrics.
        """
        # Calculate Bollinger Bands
        upperband, middleband, lowerband = talib.BBANDS(
            data['close'], timeperiod=period, nbdevup=std, nbdevdn=std, matype=0
        )

        # Generate signals
        data['signal'] = np.where(data['close'] < lowerband, 1,
                                  np.where(data['close'] > upperband, -1, 0))
        data['returns'] = data['close'].pct_change() * data['signal'].shift(1)

        # Calculate metrics
        sharpe = self.calculate_sharpe(data['returns'])
        max_drawdown = self.calculate_max_drawdown(data['returns'])
        expectancy = self.calculate_expectancy(data['returns'])

        return {
            'Ticker': ticker,
            'Best_Period': period,
            'Best_Std': std,
            'Sharpe': sharpe,
            'Max_Drawdown': max_drawdown,
            'Expectancy': expectancy
        }

    @staticmethod
    def calculate_sharpe(returns, risk_free_rate=0.025):
        """
        Calculate Sharpe Ratio.
        """
        mean_return = returns.mean()
        std_dev = returns.std()
        if std_dev == 0:
            return 0
        return (mean_return - risk_free_rate) / std_dev

    @staticmethod
    def calculate_max_drawdown(returns):
        """
        Calculate Max Drawdown.
        """
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.cummax()
        drawdown = running_max - cumulative
        return drawdown.max()

    @staticmethod
    def calculate_expectancy(returns):
        """
        Calculate Expectancy.
        """
        wins = returns[returns > 0]
        losses = returns[returns < 0]
        win_rate = len(wins) / len(returns) if len(returns) > 0 else 0
        loss_rate = 1 - win_rate
        avg_win = wins.mean() if len(wins) > 0 else 0
        avg_loss = losses.mean() if len(losses) > 0 else 0
        return (win_rate * avg_win) - (loss_rate * avg_loss)



# # Instanciar e otimizar
# optimizer = ParamsOptimization(ticker=["AAPL", "GOOG"], market_type="stocks", timeframe="1d")
# best_params = optimizer.optimize()

# # Exibir resultados
# print(best_params)

## GRAPHS

In [15]:
import talib
import pandas as pd

class TrendMetrics():
    """
    A class that encapsulates technical analysis metrics using TA-Lib.
    """

    def __init__(self):
        self.result_df = pd.DataFrame(columns=['function', 'signal'])
        self.crossover_info = pd.DataFrame(columns=['function', 'signal', 'period_low', 'period_mid', 'period_high', 'ema_low', 'ema_mid', 'ema_high'])
        self.sma_bands_info = pd.DataFrame(columns=['function', 'signal', 'period', 'std', 'lower_band', 'middle_band', 'upper_band'])
        self.rsi_info = pd.DataFrame(columns=['function', 'signal', 'period', 'upper_level', 'lower_level'])

    def get_crossover(self, data: pd.DataFrame, l1: int, l2: int, l3: int):
        """
        This function measures the crossover of 3 EMAs using TA-Lib.
        
        Parameters:
        - data: DataFrame containing the price data with a 'close' column.
        - l1, l2, l3: Periods for the 3 EMAs.
        
        Returns:
        - Updates self.crossover_signal with 'Buy', 'Sell', or 'Flat'.
        """
        period_low, period_mid, period_high = l1, l2, l3
        
        # Compute EMAs
        ema1 = talib.EMA(data['Close'], timeperiod=l1)
        ema2 = talib.EMA(data['Close'], timeperiod=l2)
        ema3 = talib.EMA(data['Close'], timeperiod=l3)

        # Current values
        ema_low, ema_mid, ema_high = ema1.iloc[-1], ema2.iloc[-1], ema3.iloc[-1]

        # Determine signal
        if ema_low > ema_mid > ema_high:
            crossover_signal = 'Buy'
        elif ema_low < ema_mid < ema_high:
            crossover_signal = 'Sell'
        else:
            crossover_signal = 'Flat'

        # Save result
        self.result_df = pd.concat([self.result_df, pd.DataFrame({
            'function': ['Crossover'],
            'signal': [crossover_signal]
        })], ignore_index=True)

        # Save result
        self.crossover_info = pd.concat([self.crossover_info, pd.DataFrame({
            'function': ['Crossover'],
            'signal': [crossover_signal],
            'period_low': [period_low],
            'period_mid': [period_mid],
            'period_high': [period_high],
            'ema1_now': [ema_low],
            'ema2_now': [ema_mid],
            'ema3_now': [ema_high],
        })], ignore_index=True)

    def get_sma_bands(self, data: pd.DataFrame, length: int=15, std_dev: int = 1):
        """
        This function calculates Bollinger Bands and detects signals based on them.
        
        Parameters:
        - data: DataFrame containing the price data with a 'close' column.
        - length: SMA period.
        - std_dev: Number of standard deviations for the bands.
        
        Returns:
        - Updates self.bbands_signal with 'Buy', 'Sell', or 'Flat'.
        """

        period, std = length, std_dev

        # Compute Bollinger Bands
        upper_band, middle_band, lower_band = talib.BBANDS(
            data['Close'], timeperiod=length, nbdevup=std_dev, nbdevdn=std_dev, matype=0
        )

        # Current values
        last_close = data['Close'].iloc[-1]
        lower_band, middle_band, upper_band = lower_band.iloc[-1], middle_band.iloc[-1], upper_band.iloc[-1]

        # Determine signal
        if last_close <= lower_band:
            bbands_signal = 'Buy'
        elif last_close >= upper_band:
            bbands_signal = 'Sell'
        else:
            bbands_signal = 'Flat'

        # Save result
        self.result_df = pd.concat([self.result_df, pd.DataFrame({
            'function': ['Bollinger_Bands'],
            'signal': [bbands_signal]
        })], ignore_index=True)
        
        self.sma_bands_info = pd.concat([self.sma_bands_info, pd.DataFrame({
            'function': ['Bollinger_Bands'],
            'signal': [bbands_signal],
            'period': [period],
            'std': [std],
            'lower_band': [lower_band],
            'middle_band': [middle_band],
            'upper_band': [upper_band]
        })], ignore_index=True)


    def get_rsi(self, data: pd.DataFrame, length: int = 25, overbought: int = 70, oversold: int = 30):
        """
        This function calculates the RSI and generates a signal based on overbought/oversold levels.
        
        Parameters:
        - data: DataFrame containing the price data with a 'close' column.
        - length: RSI period.
        - overbought: RSI overbought threshold.
        - oversold: RSI oversold threshold.
        
        Returns:
        - Updates self.rsi_signal with 'Buy', 'Sell', or 'Flat'.
        """

        period, upper_level, lower_level = length, overbought, oversold 

        # Compute RSI
        rsi = talib.RSI(data['Close'], timeperiod=length)
        rsi_now = rsi.iloc[-1]

        # Determine signal
        if rsi_now >= overbought:
            rsi_signal = 'Sell'
        elif rsi_now <= oversold:
            rsi_signal = 'Buy'
        else:
            rsi_signal = 'Flat'

        # Save result
        self.result_df = pd.concat([self.result_df, pd.DataFrame({
            'function': ['RSI'],
            'signal': [rsi_signal]
        })], ignore_index=True)

        self.rsi_info = pd.concat([self.rsi_info, pd.DataFrame({
            'function': ['RSI'],
            'signal': [rsi_signal],
            'period': [period],
            'upper_level': [upper_level],
            'lower_level': [lower_level]
        })], ignore_index=True)


In [16]:
import warnings
import yfinance as yf
from datetime import datetime

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


class DataHistory():

    def __init__(self) -> None:
        self
    
    def get_yahoo_data_history(self, symbol : str, period : str, interval : str, start = '1900-01-01', end = datetime.now(), prepost : bool = True):
        '''
        Data collection from yahoo

        Parameters:
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
        end: str
            Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
        prepost : bool
            Include Pre and Post market data in results? Default is False
        '''
        yahoo_data_history = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost)
        return yahoo_data_history
    
    def get_yahoo_symbol_info(self, symbol : str):
        '''
        Return detailed information about asset
        '''
        yahoo_symbol_info = yf.Ticker(symbol).info
        return yahoo_symbol_info
    
    def get_yahoo_symbol_dividends(self, symbol : str):
        '''
        Return dividents historics
        '''
        yahoo_symbol_dividends = yf.Ticker(symbol).dividends
        return yahoo_symbol_dividends
    
    def get_yahoo_symbol_splits(self, symbol : str):
        '''
        Return actions splits historics
        '''
        yahoo_symbol_splits = yf.Ticker(symbol).splits
        return yahoo_symbol_splits
 
    def get_yahoo_symbol_recommendations(self, symbol : str):
        '''
        Return recommendations about asset
        '''
        yahoo_symbol_recommendations = yf.Ticker(symbol).recommendations
        return yahoo_symbol_recommendations

    def get_yahoo_symbol_calendar(self, symbol : str):
        '''
        Return corporative calendar events about asset
        '''
        yahoo_symbol_calendar = yf.Ticker(symbol).calendar
        return yahoo_symbol_calendar

    def get_yahoo_symbol_major_holders(self, symbol : str):
        '''
        Return the list of major holders
        '''
        yahoo_symbol_major_holders = yf.Ticker(symbol).major_holders
        return yahoo_symbol_major_holders

    def get_yahoo_symbol_institutional_holders(self, symbol : str):
        '''
        Return the list of major institutional holders
        '''
        yahoo_symbol_institutional_holders = yf.Ticker(symbol).institutional_holders
        return yahoo_symbol_institutional_holders

    def get_yahoo_symbol_balance_sheet(self, symbol : str):
        '''
        Return the patrimonial balance sheet
        '''
        yahoo_symbol_balance_sheet = yf.Ticker(symbol).balance_sheet
        return yahoo_symbol_balance_sheet

    def get_yahoo_symbol_financials(self, symbol : str):
        '''
        !!! Not Working !!!
        Return the financials results (profits and expenses)
        '''
        yahoo_symbol_financials = yf.Ticker(symbol).financials
        return yahoo_symbol_financials

    def get_yahoo_symbol_cashflow(self, symbol : str):
        '''
        Return the cashflow results
        '''
        yahoo_symbol_cashflow = yf.Ticker(symbol).cashflow
        return yahoo_symbol_cashflow

    def get_yahoo_symbol_sustainability(self, symbol : str):
        '''
        
        Return the ESG metrics (enviormental, social and governamental)
        '''
        yahoo_symbol_sustainability = yf.Ticker(symbol).sustainability
        return yahoo_symbol_sustainability

    def get_yahoo_symbol_news(self, symbol : str):
        '''
        Return the latest news about asset
        '''
        yahoo_symbol_news = yf.Ticker(symbol).news
        return yahoo_symbol_news

    def get_yahoo_symbol_fast_info(self, symbol : str):
        '''
        Return the fast information about asset

        Data:
        exchange : str
            Exchange on which the asset is traded
        marketCap : float
            Marker Cap of asset
        quoteType: str
            Asset type (EQUITY, CRYPTO, FOREX..)
        shares : int
            Total Number of shares in circulation            
        '''
        yahoo_symbol_fast_info_exchange = yf.Ticker(symbol).fast_info.exchange
        yahoo_symbol_fast_info_marketcap = yf.Ticker(symbol).fast_info.market_cap
        yahoo_symbol_fast_info_quotetype = yf.Ticker(symbol).fast_info.quote_type
        yahoo_symbol_fast_info_shares = yf.Ticker(symbol).fast_info.shares
        return yahoo_symbol_fast_info_exchange, yahoo_symbol_fast_info_marketcap, yahoo_symbol_fast_info_quotetype, yahoo_symbol_fast_info_shares
    




In [17]:
dh = DataHistory()
df = dh.get_yahoo_data_history('MSFT', '1y', '1d')

tm = TrendMetrics()
tm.get_crossover(data=df, l1=25, l2=50, l3=200 )
tm.get_sma_bands(data=df, length=15, std_dev=1)
tm.get_rsi(data=df, length=25, overbought=70, oversold=30)

print('------------- Results df -------------\n\n',tm.result_df,'\n\n------------- ')

print('------------- Results Crossover -------------\n\n',tm.crossover_info,'\n\n------------- ')

print('------------- Results Bollinger Bands -------------\n\n',tm.sma_bands_info,'\n\n------------- ')

print('------------- Results RSI -------------\n\n',tm.rsi_info,'\n\n------------- ')

------------- Results df -------------

           function signal
0        Crossover    Buy
1  Bollinger_Bands    Buy
2              RSI   Flat 

------------- 
------------- Results Crossover -------------

     function signal period_low period_mid period_high ema_low ema_mid  \
0  Crossover    Buy         25         50         200     NaN     NaN   

  ema_high    ema1_now    ema2_now    ema3_now  
0      NaN  432.187188  430.023366  417.012547   

------------- 
------------- Results Bollinger Bands -------------

           function signal period std  lower_band  middle_band  upper_band
0  Bollinger_Bands    Buy     15   1  425.586317   436.358663  447.131009 

------------- 
------------- Results RSI -------------

   function signal period upper_level lower_level
0      RSI   Flat     25          70          30 

------------- 


C:\Users\joaom\AppData\Local\Temp\ipykernel_8728\1517949193.py:100: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.sma_bands_info = pd.concat([self.sma_bands_info, pd.DataFrame({


In [1]:
from PySide6.QtCore import Qt
from PySide6.QtGui import QColor
